In [ ]:
import cv2
import json

from os import listdir, path

VIDEO_PATH = "../../vids/0801-1152"
CAMERA_DB_PATH = "./metadata/cameras.json"
VIDEO_DB_PATH = "./metadata/videos.json"
METADATA_PATH = "./metadata/0801-1152"

In [ ]:
with open(CAMERA_DB_PATH, "r") as f:
  camera_data = json.load(f)

with open(VIDEO_DB_PATH, "r") as f:
  video_data = json.load(f)

### Resolution

In [ ]:
for io_dir in sorted(camera_data.keys()):
  input_dir_path = path.join(VIDEO_PATH, io_dir)
  input_files = sorted([f for f in listdir(input_dir_path) if f.endswith("mp4")])

  m_cam_res = camera_data[io_dir]["resolution"]
  cam_width = m_cam_res[0] if 0 < len(m_cam_res) else 0
  cam_height = m_cam_res[1] if 1 < len(m_cam_res) else 0

  for io_file in input_files:
    print(io_file)
    input_file_path = path.join(input_dir_path, io_file)
    vid = cv2.VideoCapture(input_file_path)

    m_width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    m_height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))

    if cam_width == 0 or cam_height == 0:
      cam_width, cam_height = m_width, m_height
    elif abs(cam_width - m_width) > 2:
      print("ERROR: video resolution", io_file, cam_width, m_width)
    elif abs(cam_height - m_height) > 2:
      print("ERROR: video resolution", io_file, cam_height, m_height)
    else:
      cam_width, cam_height = m_width, m_height

    vid.release()

  camera_data[io_dir]["resolution"] = [cam_width, cam_height]

### FPS

In [ ]:
for io_dir in sorted(camera_data.keys()):
  print(io_dir)
  input_dir_path = path.join(VIDEO_PATH, io_dir)
  input_files = sorted([f for f in listdir(input_dir_path) if f.endswith("mp4")])

  vid_fps = {}

  for io_file in input_files:
    input_file_path = path.join(input_dir_path, io_file)
    vid = cv2.VideoCapture(input_file_path)
    m_fps = round(vid.get(cv2.CAP_PROP_FPS))

    vid_fps[m_fps] = m_fps
    video_data[io_file]["fps"] = m_fps

    meta_file_path = path.join(METADATA_PATH, io_dir, io_file.replace("mp4", "json"))
    with open(meta_file_path, "r+") as f:
      m_video_data = json.load(f)
      m_video_data["fps"] = m_fps
      f.seek(0)
      json.dump(m_video_data, f)
      f.truncate()

    vid.release()

  camera_data[io_dir]["fps"] = sorted(list(vid_fps.keys()))

In [ ]:
with open(CAMERA_DB_PATH, "w") as f:
  json.dump(camera_data, f, indent=2)

with open(VIDEO_DB_PATH, "w") as f:
  json.dump(video_data, f, indent=2, separators=(',',':'))